Q1. Load the dataset (5 points)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re
import unicodedata
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
import numpy as np
import pandas as pd

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/blogtext.csv')

In [5]:
df.shape

(681284, 7)

In [6]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


I am taking 5000 data only. As it is  not working.

In [0]:
df=df[0:5000]

**Q2. Preprocess rows of the “text” column

In [0]:
df=df.drop(['id','date'],axis=1)

In [0]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [10]:
df['text']

0                  Info has been found (+/- 100 pages,...
1                  These are the team members:   Drewe...
2                  In het kader van kernfusie op aarde...
3                        testing!!!  testing!!!          
4                    Thanks to Yahoo!'s Toolbar I can ...
                              ...                        
4995           So... I had another one of those dreams...
4996           mmm... strawberry tea for breakfast. To...
4997           Yay for a new layout!!  Yeah, I know, I...
4998           Ok, so I lied... Fed up isn't playing F...
4999           well, today I went to church and talked...
Name: text, Length: 5000, dtype: object

In [0]:
df['text_new']=df['text'].str.lower()

In [0]:
def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [0]:
def lemmatize_text(text):

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])  

In [0]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [0]:
def normalize_corpus(corpus, html_stripping=True, accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # remove special characters and\or digits    
        if special_char_removal:
        # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)  
        # remove extra whitespace
            doc = re.sub(' +', ' ', doc)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [0]:
df['text_new'] =normalize_corpus(df['text_new'])

In [17]:
df.head()

,gender,age,topic,sign,text,text_new
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info ha been found page and mb of pdf file now...
1,male,15,Student,Leo,These are the team members: Drewe...,these are the team member drewes van der laag ...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,in het kader van kernfusie op aarde maak je ei...
3,male,15,Student,Leo,testing!!! testing!!!,testing testing
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks to yahoo s toolbar i can now capture th...


In [18]:
df.shape

(5000, 6)

In [19]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re
import unicodedata
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
df.shape

(5000, 6)

**Q3. As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence.

In [0]:
df['label'] =df["gender"].map(str)+ ', ' +df["age"].map(str)+ ', ' +df["topic"].map(str)+ ', ' +df["sign"]

In [22]:
df.head(5)

,gender,age,topic,sign,text,text_new,label
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info ha been found page and mb of pdf file now...,"male, 15, Student, Leo"
1,male,15,Student,Leo,These are the team members: Drewe...,these are the team member drewes van der laag ...,"male, 15, Student, Leo"
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,in het kader van kernfusie op aarde maak je ei...,"male, 15, Student, Leo"
3,male,15,Student,Leo,testing!!! testing!!!,testing testing,"male, 15, Student, Leo"
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks to yahoo s toolbar i can now capture th...,"male, 33, InvestmentBanking, Aquarius"


In [0]:
data=df[['text_new','label']]

In [24]:
data.head(5)

,text_new,label
0,info ha been found page and mb of pdf file now...,"male, 15, Student, Leo"
1,these are the team member drewes van der laag ...,"male, 15, Student, Leo"
2,in het kader van kernfusie op aarde maak je ei...,"male, 15, Student, Leo"
3,testing testing,"male, 15, Student, Leo"
4,thanks to yahoo s toolbar i can now capture th...,"male, 33, InvestmentBanking, Aquarius"


In [25]:
data['label'].nunique()

96

In [26]:
data.astype

<bound method NDFrame.astype of                                                text_new                                  label
0     info ha been found page and mb of pdf file now...                 male, 15, Student, Leo
1     these are the team member drewes van der laag ...                 male, 15, Student, Leo
2     in het kader van kernfusie op aarde maak je ei...                 male, 15, Student, Leo
3                                       testing testing                 male, 15, Student, Leo
4     thanks to yahoo s toolbar i can now capture th...  male, 33, InvestmentBanking, Aquarius
...                                                 ...                                    ...
4995  so i had another one of those dream last night...            female, 17, indUnk, Scorpio
4996  mmm strawberry tea for breakfast tomorrow i th...            female, 17, indUnk, Scorpio
4997  yay for a new layout yeah i know i need to get...            female, 17, indUnk, Scorpio
4998  ok so i lied

**Q4. Separate features and labels, and split the data into training and testing

In [27]:
X=data['text_new']
Y=data['label']
print(X.shape)
print(Y.shape)

(5000,)
(5000,)


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=2)

In [30]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3750,) (3750,)
(1250,) (1250,)


**Q5. Vectorize the features

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 2),stop_words='english')

In [32]:
cv.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [33]:
len(cv.vocabulary_)

228837

In [0]:
x_train_dtm=cv.transform(X_train)

In [0]:
x_test_dtm=cv.transform(X_test)

In [36]:
x_train_dtm[0]

<1x228837 sparse matrix of type '<class 'numpy.int64'>'
	with 104 stored elements in Compressed Sparse Row format>

In [37]:
x_test_dtm[0]

<1x228837 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [38]:
print(x_train_dtm.shape)
print(x_test_dtm.shape)

(3750, 228837)
(1250, 228837)


In [39]:
x_train_dtm

<3750x228837 sparse matrix of type '<class 'numpy.int64'>'
	with 437929 stored elements in Compressed Sparse Row format>

In [40]:
x_test_dtm

<1250x228837 sparse matrix of type '<class 'numpy.int64'>'
	with 85708 stored elements in Compressed Sparse Row format>

**Q7. Transform the labels

In [41]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=Y_train.unique())
mlb.fit_transform('Y_train')
mlb.transform('Y_test')

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Y', '_', 'a', 'i', 'n', 'r', 't'] will be ignored
  .format(sorted(unknown, key=str)))
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Y', '_', 'e', 's', 't'] will be ignored
  .format(sorted(unknown, key=str)))


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0,

In [42]:
x_train_dtm.shape

(3750, 228837)

In [43]:
print(Y_train.shape)
print(Y_test.shape)

(3750,)
(1250,)


**Q8. Choose a classifier - (5 points)**

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs',max_iter=2000)
clf1vR = OneVsRestClassifier(clf)

**Q9. Fit the classifier, make predictions and get the accuracy (5 points)**

In [45]:
clf1vR.fit(x_train_dtm,Y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
predicted=clf1vR.predict(x_test_dtm)

In [47]:
from sklearn.metrics import accuracy_score
print( "Accuracy Score: ",accuracy_score(Y_test, predicted))

Accuracy Score:  0.6928


**Q10. Print true label and predicted label for any five examples 


In [48]:
print(Y_train,predicted)

4715     male, 25, Technology, Aries
3576     male, 35, Technology, Aries
4996     female, 17, indUnk, Scorpio
2556     male, 35, Technology, Aries
611     male, 24, Engineering, Libra
                    ...             
3335     male, 35, Technology, Aries
1099      female, 15, Student, Libra
2514     male, 35, Technology, Aries
3606     male, 35, Technology, Aries
2575     male, 35, Technology, Aries
Name: label, Length: 3750, dtype: object ['male, 35, Technology, Aries' 'female, 34, indUnk, Sagittarius'
 'male, 35, Technology, Aries' ... 'male, 35, Technology, Aries'
 'male, 39, Communications-Media, Libra' 'female, 34, indUnk, Sagittarius']
